In [1]:
import pandas as pd 
import numpy as np
from src.download import *
import zipfile 
import glob
from tqdm import tqdm  
import os
from src.files.fasta import FASTAFile
from sklearn.metrics import normalized_mutual_info_score, mutual_info_score, adjusted_mutual_info_score, completeness_score
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy.stats import entropy 
import re
import subprocess
import math 
from utils import * 
from src.data import * 

%load_ext autoreload
%autoreload 2

niks_domain = '[XNGDRYS]IK[SDLT]'
# ycf_domain = 'Y.C...F'

In [6]:
arf1_df = pd.read_csv('../data/arf1_duplicates.csv', index_col=0)

def load_msa(path:str='../data/arf1_cleaned.afa', ids:list=None, conservation_threshold=0.8):
    df = FASTAFile().from_fasta(path).to_df()
    df = df.loc[ids].copy()
    alignment = np.array([list(seq) for seq in df.seq])

    # Flag conserved positions as those where at least 80 percent of the sequences do not have a gap. 
    is_conserved = lambda col : (col != '-').astype(int).mean() > conservation_threshold

    conserved_positions = np.where([is_conserved(col) for col in alignment.T])[0]
    print('load_msa: Num. conserved positions:', len(conserved_positions))
    alignment = alignment[:, conserved_positions].copy() # Ignore the extra junk in the alignment. 
    return df.index.values, alignment

index, alignment = load_msa(ids=arf1_df.index)

load_msa: Num. conserved positions: 413


In [7]:
seq = ''.join(alignment[0]) # The first entry in the alignment has the canonical NIKS domain and YXCXXXF.

niks_match = re.search('NIKS', seq)
# ycf_match = re.search(ycf_domain, seq, flags=re.DOTALL)

start, stop = niks_match.start(), niks_match.end()

In [8]:
seq

'MTEAHEKYEFKKKLESLRDKKGRSTELISLYIPADKQIFDVTNQLKDEHGQAANIKSKLTRTNVQGAIESLLSRLRYLDKVPENGIVYFTGAVDIGANKTSMESEVIIPPEPITVYKYHCDSSFYLEPLEDMLKDKSTFGLLVLDRREATIGLLVGKRIQAFRNLTSTVPGKQRKGGQSAHRFQQLRLIAIHDFYKRIGDAASEIFMAVHKDLKGVLIGGPSPTKEEFYGGEFLHHELQKKILGLFDTAYTDESGLSELVNAAGEKLQDLELMGQKNAVRDFFKELIADSGKVAYGETQVRANLEINAVDVLLLSEDLRAERVTTKCSVCGYENKWTRRWKPGEPAPTAGNCPKCGSSLEVTDVIDVVDEFSELADKSNAKVVFVSTDFDEGSQLMNAFGGIAAILRYSTGV*'

In [ ]:
# domain = 'niks'

# def figure(arf1_df:pd.DataFrame, domain:str='niks'):
#     fig, (ax_top, ax_bottom) = get_split_figure((0, 500), (2200, 2400))

#     # figure_df = pd.DataFrame(index=pd.Series(np.arange(1, 4), name='num_arf1'))
#     figure_df = list()

#     categories = ['pyl', f'pyl_{domain}', 'no_pyl', f'no_pyl_{domain}']
#     masks = [(arf1_df.has_pyl), (arf1_df.has_pyl & arf1_df[f'has_{domain}_domain']), (~arf1_df.has_pyl), (~arf1_df.has_pyl & arf1_df[f'has_{domain}_domain'])]
#     palette = {'pyl':'darkseagreen', f'pyl_{domain}':'seagreen', 'no_pyl':'indianred', f'no_pyl_{domain}':'firebrick'}

#     for category, mask in zip(categories, masks):
#         num_arf1_counts = arf1_df[mask].groupby('genome_id').size().value_counts()
#         figure_df += [{'num_arf1':n, 'count':num_arf1_counts.loc[n], 'category':category} for n in num_arf1_counts.index]
#     figure_df = pd.DataFrame(figure_df)
#     figure_df = figure_df[figure_df.num_arf1 < 4].copy()

#     # How many cases of Pyl-containing organisms with two ARF-1s and only one with a NIKS domain?
#     arf1_counts = arf1_df.groupby('genome_id').size()
#     genome_ids_multiple_arf1s = arf1_counts[arf1_counts > 1].index.values.tolist()
#     # no_pyl_genome_ids_multiple_arf1s = arf1_counts[(arf1_counts > 1) & (arf1_counts.index.isin(no_pyl_genome_ids))].index.values.tolist()

#     sns.barplot(figure_df, x='num_arf1', y='count', hue='category', edgecolor='black', ax=ax_top, palette=palette) #, binsize=0.2)
#     sns.barplot(figure_df, x='num_arf1', y='count', hue='category', edgecolor='black', ax=ax_bottom, legend=False, palette=palette) #, binsize=0.2)
#     ax_top.set_ylabel('')
#     ax_top.get_legend().set_title('')
#     ax_bottom.set_xlabel('num. aRF-1')

#     plt.show()
#     return genome_ids_multiple_arf1s

# genome_ids_multiple_arf1s = figure(arf1_df)
# arf1_df['has_multiple_arf1'] = arf1_df.genome_id.isin(genome_ids_multiple_arf1s)


# # It's concerning that some of the non-Pyl organisms have multiple aRF-1 genes. None of the non-Pyl genomes with multiple
# # aRF-1s have a lot of contamination, so it's possible that they have Pyl machinery that were missed by the HMMs.